In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')
import time 
from tqdm import tqdm

In [2]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument("--single-process")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome('C:/Users/YONSAI/Downloads/chromedriver', options=options)

songIds = []
for i in tqdm(range(1964, 2023)):
    url = f'https://www.melon.com/chart/age/index.htm?chartType=YE&chartGenre=KPOP&chartDate={i}' 
    driver.get(url)
    time.sleep(2)

    soup = BeautifulSoup(driver.page_source,'html.parser')
    trs = soup.select('tbody > tr.lst50')

    for tr in trs:
        if tr.select_one('.bullet_icons.age_19'):
            continue
        songId  = tr.select_one('.btn.btn_icon_detail')['onclick'].split("'")[-2]
        songIds.append(songId)
songIds[:5],len(songIds)

100%|██████████| 59/59 [03:06<00:00,  3.17s/it]


(['4083218', '1003023', '1754987', '1002782', '1003568'], 2646)

In [ ]:
# 시대별 1964 ~ 2022 (59년)
# 2646곡에서 중복제거시 2544
# 300곡씩 파일 저장하기
# songIds가 1964~ 2022까지  set 전 2646
# set(songIds) = 2544개  
# songIds[:300]     (247, 247)
# songIds[300:600]  (244, 242)
# songIds[600:900]  (257, 253)
# songIds[900:1200] (272, 258)
# songIds[1200:1500]    (273, 268)
# songIds[1500:1800]    (283, 281)
# songIds[1800:2100]    (288, 287)
# songIds[2100:2400]    (298, 284)
# songIds[2400:]    (245, 206)

In [35]:
infos = []
for songId in tqdm(songIds[2400:]):
    try:
        song_url = f'https://www.melon.com/song/detail.htm?songId={songId}'
        driver.get(song_url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        info = soup.select_one('#conts')
        # songId = info.select_one('#btnLike')['data-song-no']
        title = info.select_one('.song_name').get_text().strip().split('\t')[-1]
        artist = info.select_one('.artist_name').get_text()
        genre = info.select_one('dl > dd:nth-child(6)').get_text().replace(',', '')
        album = info.select_one('.list > dd').get_text()
        ul = info.select('.list_person.clfix > li')
        lyricist = ul[0].select_one('.artist_name').get_text().strip() # 작곡가 스트립
        composer = ul[1].select_one('.artist_name').get_text()
        date = info.select_one('dl > dd:nth-child(4)').get_text().replace('.','')
        img = info.select_one('.image_typeAll > img')['src']
        comment = info.select_one('#revCnt').get_text().replace('개', '').replace(',','')  # 콤마없어야되고
        like = info.select_one('#d_like_count').get_text().replace(',', '')
        lyric = str(info.select_one('#d_video_summary')).split('\t\t\t')[-1].replace('\t', '').replace('<br/>', '\n').split('</div>')[0].strip()
        try:
            buttons = soup.select('.meta > button')
            plylstSeq = ' '.join([button['data-djcol-no'] for button in buttons])
            infos.append({'songId': songId, 'title': title, 'artist': artist, 'genre':genre,
                        'album': album, 'lyricist': lyricist, 'composer': composer, 'date':date, 'img': img,
                        'comment': comment, 'like': like, 'lyric': lyric, 'plylstSeq': plylstSeq})
        except:
            plylstSeq = np.nan
            infos.append({'songId': songId, 'title': title, 'artist': artist, 'genre':genre,
                        'album': album, 'lyricist': lyricist, 'composer': composer, 'date':date, 'img': img,
                        'comment': comment, 'like': like, 'lyric': lyric, 'plylstSeq': plylstSeq})

    except:
        continue

100%|██████████| 246/246 [15:54<00:00,  3.88s/it]


In [36]:
df = pd.DataFrame(infos)
df.tail(2)

,songId,title,artist,genre,album,lyricist,composer,date,img,comment,like,lyric,plylstSeq
243,34819473,사건의 지평선,윤하 (YOUNHA),록/메탈,YOUNHA 6th Album Repackage 'END THEORY : Final...,윤하 (YOUNHA),윤하 (YOUNHA),20220330,https://cdnimg.melon.co.kr/cm2/album/images/10...,"3,142",302118,,NaN
244,34494149,Step Back,GOT the beat,댄스,Step Back,유영진,유영진,20220103,https://cdnimg.melon.co.kr/cm2/album/images/10...,"1,417",98139,You must step back\n어델 어델 봐\n너 감히 누구라고 날 제껴\n이...,NaN


In [37]:
df.shape[0], df.songId.nunique()

(245, 206)

In [38]:
df.to_csv('data/melon_year9.csv', index=False, encoding='utf-8')
pd.read_csv('data/melon_year9.csv').head(2)

,songId,title,artist,genre,album,lyricist,composer,date,img,comment,like,lyric,plylstSeq
0,30816860,뿜뿜,모모랜드 (MOMOLAND),댄스,GREAT!,신사동 호랭이,범이,20180103,https://cdnimg.melon.co.kr/cm/album/images/101...,"1,428",150387,설레어 너와 나의 랑데뷰 \n랑데뷰\n내 마음을 들었다 놨다 해 맘대루 \n맘대루\...,NaN
1,31131273,뚜두뚜두 (DDU-DU DDU-DU),BLACKPINK,랩/힙합,SQUARE UP,TEDDY,TEDDY,20180615,https://cdnimg.melon.co.kr/cm/album/images/101...,"2,240",257646,착한 얼굴에 그렇지 못한 태도\n가녀린 몸매 속\n가려진 volume은 두 배로\n...,NaN


In [28]:
pd.read_csv('data/melon_year9.csv').head(110).tail(10)

,songId,title,artist,genre,album,lyricist,composer,date,img,comment,like,lyric,plylstSeq
100,32578498,에잇(Prod.&Feat. SUGA of BTS),아이유,록/메탈,에잇,아이유,SUGA,20200506,https://cdnimg.melon.co.kr/cm2/album/images/10...,"10,042",392173,NaN,NaN
101,32183386,Blueming,아이유,록/메탈,Love poem,아이유,이종훈,20191118,https://cdnimg.melon.co.kr/cm2/album/images/10...,"2,732",354161,‘뭐해?‘라는 두 글자에\r\n‘네가 보고 싶어’ 나의 속마음을 담아 우\r\n이모...,NaN
102,32156286,늦은 밤 너의 집 앞 골목길에서,노을,발라드,늦은 밤 너의 집 앞 골목길에서,미후왕,빅가이로빈 (bigguyrobin),20191107,https://cdnimg.melon.co.kr/cm2/album/images/10...,"1,465",208185,오래되었어 그때 너의 웃음\r\n너의 목소리 잊혀진거 같아\r\n시간의 위로 그 말...,NaN
103,32345931,시작,가호 (Gaho),록/메탈 국내드라마,이태원 클라쓰 OST Part.2,서동성,박성일,20200201,https://cdnimg.melon.co.kr/cm2/album/images/10...,"2,333",246322,새로운 시작은 늘 설레게 하지\r\n모든 걸 이겨낼 것처럼\r\n시간을 뒤쫓는 시계...,NaN
104,31737197,작은 것들을 위한 시 (Boy With Luv) (Feat. Halsey),방탄소년단,댄스,MAP OF THE SOUL : PERSONA,Pdogg,RM,20190412,https://cdnimg.melon.co.kr/cm/album/images/102...,"8,711",442186,모든 게 궁금해\r\nHow’s your day\r\nOh tell me\r\n뭐가...,NaN
105,32273582,Psycho,Red Velvet (레드벨벳),댄스,‘The ReVe Festival’ Finale,KENZIE,Andrew Scott,20191223,https://cdnimg.melon.co.kr/cm2/album/images/10...,"5,696",239456,NaN,NaN
106,32061975,"어떻게 이별까지 사랑하겠어, 널 사랑하는 거지",AKMU (악뮤),발라드,항해,이찬혁,이찬혁,20190925,https://cdnimg.melon.co.kr/cm2/album/images/10...,"2,456",404854,일부러 몇 발자국 물러나\r\n내가 없이 혼자 걷는 널 바라본다\r\n옆자리 허전한...,NaN
107,32872978,Dynamite,방탄소년단,댄스,Dynamite (DayTime Version),David Stewart,Jessica Agombar,20200824,https://cdnimg.melon.co.kr/cm2/album/images/10...,"19,099",451983,Cos ah ah\r\nI’m in the stars tonight\r\nSo wa...,NaN
108,32378104,마음을 드려요,아이유,발라드 국내드라마,사랑의 불시착 OST Part 11,남혜승,박진호,20200215,https://cdnimg.melon.co.kr/cm2/album/images/10...,"1,800",245577,NaN,411111859 436119495
109,32486613,처음처럼,엠씨더맥스 (M.C the MAX),발라드,CEREMONIA,이수 (엠씨더맥스),CR KIM,20200325,https://cdnimg.melon.co.kr/cm2/album/images/10...,"3,045",115431,눈이 부시게 웃던 널 기억해 \r\n이제는 다시 볼 수 없게 됐지만 \r\n끝내 붙...,NaN
